<a href="https://colab.research.google.com/github/tenochca/DataScienceProject/blob/main/finalproject_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_diabetes = pd.read_csv('/content/drive/MyDrive/DSC 343/diabetes_prediction_dataset.csv')
df_diabetes.head(5)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
#drop duplicates
df_diabetes.drop_duplicates(inplace = True)
df_diabetes.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
col_to_move = df_diabetes.pop('diabetes')
df_diabetes.insert(0, 'diabetes', col_to_move)
#moving diabetes to the first column

In [ ]:
#putting the bmi column into bins
bmi_bins = pd.qcut(df_diabetes.bmi, 3)
bmi_bins.head(10)

0    (10.009, 25.3]
1     (25.3, 27.73]
2     (25.3, 27.73]
3    (10.009, 25.3]
4    (10.009, 25.3]
5     (25.3, 27.73]
6    (10.009, 25.3]
7    (10.009, 25.3]
8    (27.73, 95.69]
9     (25.3, 27.73]
Name: bmi, dtype: category
Categories (3, interval[float64, right]): [(10.009, 25.3] < (25.3, 27.73] < (27.73, 95.69]]

In [ ]:
#specify quantiles for discretization, the reange for a nomral bmi is low compared to the range of the data
bmi_bins = pd.qcut(df_diabetes.bmi, [0, 0.2, 0.3, 1], labels=["low", "norm", "high"])
bmi_bins.head(10)

0    high
1    high
2    high
3    norm
4     low
5    high
6     low
7    norm
8    high
9    high
Name: bmi, dtype: category
Categories (3, object): ['low' < 'norm' < 'high']

In [ ]:
df_diabetes["bmi-bins"] = bmi_bins
df_diabetes.head(10)
#adding new column to dataframe

,diabetes,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,bmi-bins
0,0,Female,80.0,0,1,never,25.19,6.6,140,high
1,0,Female,54.0,0,0,No Info,27.32,6.6,80,high
2,0,Male,28.0,0,0,never,27.32,5.7,158,high
3,0,Female,36.0,0,0,current,23.45,5.0,155,norm
4,0,Male,76.0,1,1,current,20.14,4.8,155,low
5,0,Female,20.0,0,0,never,27.32,6.6,85,high
6,1,Female,44.0,0,0,never,19.31,6.5,200,low
7,0,Female,79.0,0,0,No Info,23.86,5.7,85,norm
8,0,Male,42.0,0,0,never,33.64,4.8,145,high
9,0,Female,32.0,0,0,never,27.32,5.0,100,high


In [ ]:
#dropping the old bmi column
df_diabetes.drop(columns=['bmi'], inplace=True)
df_diabetes.head()

,diabetes,gender,age,hypertension,heart_disease,smoking_history,HbA1c_level,blood_glucose_level,bmi-bins
0,0,Female,80.0,0,1,never,6.6,140,high
1,0,Female,54.0,0,0,No Info,6.6,80,high
2,0,Male,28.0,0,0,never,5.7,158,high
3,0,Female,36.0,0,0,current,5.0,155,norm
4,0,Male,76.0,1,1,current,4.8,155,low


In [ ]:
#getting the dummies
df_diabetes = pd.get_dummies(df_diabetes)
df_diabetes.head(10)

,diabetes,age,hypertension,heart_disease,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,gender_Other,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current,bmi-bins_low,bmi-bins_norm,bmi-bins_high
0,0,80.0,0,1,6.6,140,1,0,0,0,0,0,0,1,0,0,0,1
1,0,54.0,0,0,6.6,80,1,0,0,1,0,0,0,0,0,0,0,1
2,0,28.0,0,0,5.7,158,0,1,0,0,0,0,0,1,0,0,0,1
3,0,36.0,0,0,5.0,155,1,0,0,0,1,0,0,0,0,0,1,0
4,0,76.0,1,1,4.8,155,0,1,0,0,1,0,0,0,0,1,0,0
5,0,20.0,0,0,6.6,85,1,0,0,0,0,0,0,1,0,0,0,1
6,1,44.0,0,0,6.5,200,1,0,0,0,0,0,0,1,0,1,0,0
7,0,79.0,0,0,5.7,85,1,0,0,1,0,0,0,0,0,0,1,0
8,0,42.0,0,0,4.8,145,0,1,0,0,0,0,0,1,0,0,0,1
9,0,32.0,0,0,5.0,100,1,0,0,0,0,0,0,1,0,0,0,1


In [ ]:
diabetes_names = df_diabetes.columns.values
diabetes_names

array(['diabetes', 'age', 'hypertension', 'heart_disease', 'HbA1c_level',
       'blood_glucose_level', 'gender_Female', 'gender_Male',
       'gender_Other', 'smoking_history_No Info',
       'smoking_history_current', 'smoking_history_ever',
       'smoking_history_former', 'smoking_history_never',
       'smoking_history_not current', 'bmi-bins_low', 'bmi-bins_norm',
       'bmi-bins_high'], dtype=object)

In [ ]:
#assigning the y and x variables
y = df_diabetes['diabetes']
x = df_diabetes[diabetes_names[1:]]
x.head()

,age,hypertension,heart_disease,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,gender_Other,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current,bmi-bins_low,bmi-bins_norm,bmi-bins_high
0,80.0,0,1,6.6,140,1,0,0,0,0,0,0,1,0,0,0,1
1,54.0,0,0,6.6,80,1,0,0,1,0,0,0,0,0,0,0,1
2,28.0,0,0,5.7,158,0,1,0,0,0,0,0,1,0,0,0,1
3,36.0,0,0,5.0,155,1,0,0,0,1,0,0,0,0,0,1,0
4,76.0,1,1,4.8,155,0,1,0,0,1,0,0,0,0,1,0,0


In [ ]:
#splitting training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=51)

In [ ]:
#fitting scaler to the training data using z score
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
#normalizing testing and training
x_train_zscore = scaler.transform(x_train)
x_test_zscore = scaler.transform(x_test)

In [ ]:
print(x_test_zscore)

[[ 1.07852338 -0.29089936 -0.20640655 ... -0.50176267  3.00403742
  -1.52430651]
 [ 1.07852338 -0.29089936 -0.20640655 ... -0.50176267 -0.33288533
   0.65603603]
 [ 0.41090143 -0.29089936 -0.20640655 ... -0.50176267 -0.33288533
   0.65603603]
 ...
 [-0.43475305 -0.29089936 -0.20640655 ... -0.50176267 -0.33288533
   0.65603603]
 [ 0.14385264 -0.29089936 -0.20640655 ... -0.50176267 -0.33288533
   0.65603603]
 [-0.56827744 -0.29089936 -0.20640655 ... -0.50176267 -0.33288533
   0.65603603]]


# **MODEL TESTING**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
#fitting the model to the training data
knn = KNeighborsClassifier(n_neighbors=30, metric='cosine')
knn.fit(x_train_zscore, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=30)

In [ ]:
x_pred = knn.predict(x_train_zscore)
#prediction based on training

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

precision = precision_score(y_train, x_pred)
accuracy = accuracy_score(y_train, x_pred)
recall = recall_score(y_train, x_pred)

print('Precision: {:.3f}'.format(precision))
print('Accuracy: {:.3f}'.format(accuracy))
print('Recall: {:.3f}'.format(recall))

confusion_matrix(y_train, x_pred)

Precision: 0.985
Accuracy: 0.958
Recall: 0.528


array([[70098,    53],
       [ 3194,  3571]])

In [ ]:
y_pred = knn.predict(x_test_zscore)
#prediction based on testing

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('Precision: {:.3f}'.format(precision))
print('Accuracy: {:.3f}'.format(accuracy))
print('Recall: {:.3f}'.format(recall))

confusion_matrix(y_test, y_pred)

Precision: 0.987
Accuracy: 0.957
Recall: 0.520


array([[17501,    12],
       [  824,   893]])